# Building Tokenizer

In [1]:
import urllib.request

In [2]:
url = ("https://raw.githubusercontent.com/rasbt/"
 "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
 "the-verdict.txt")

file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x2b7b38cc810>)

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character: ", len(raw_text))
print(raw_text[:98])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no


# Using re to split text into tokens

In [4]:
import re

In [5]:
text = "Hello, world. This is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
# Removing all the white spaces.

result = [item for item in result if item.strip()]
print(result)

['Hello,', 'world.', 'This', 'is', 'a', 'test.']


In [7]:
# Modifying the tokenizer to handle more things such as punctuations etc.

text = "Hello, world. iS this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
reult = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'iS', ' ', 'this', '--', '', ' ', 'a', ' ', 'test', '?', '']


## Applying this working tokenizer

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [9]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Converting Tokens into token IDs

In [10]:
# Converting the tokens from python strings to integer representations to produce
#token IDs.

# We'll create a list of all unique tokens and sort them.
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


# Implementing the full text Tokenizer class

In [12]:
class SimpleTokenizerV1:
 def __init__(self, vocab):
     self.str_to_int = vocab
     self.int_to_str = {i:s for s,i in vocab.items()}

 def encode(self, text):
     preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
     preprocessed = [
     item.strip() for item in preprocessed if item.strip()
     ]
     ids = [self.str_to_int[s] for s in preprocessed]
     return ids

 def decode(self, ids):
     text = " ".join([self.int_to_str[i] for i in ids])
    
     text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
     return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
 Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


# Adding special context tokens

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


In [16]:
# Printing the last 5 entries of the updated vocabulary.

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


## Modified tokenizer

In [17]:
class SimpleTokenizerV2:
 def __init__(self, vocab):
     self.str_to_int = vocab
     self.int_to_str = { i:s for s,i in vocab.items()}

 def encode(self, text):
     preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
     preprocessed = [
     item.strip() for item in preprocessed if item.strip()
     ]
     preprocessed = [item if item in self.str_to_int
     else "<|unk|>" for item in preprocessed]
     ids = [self.str_to_int[s] for s in preprocessed]
     return ids

 def decode(self, ids):
     text = " ".join([self.int_to_str[i] for i in ids])
     text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
     return text

In [18]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


# Data Sampling with a Sliding Window
 -  Implementing the loader wthat will create the input-target pairs from the training
 -  dataset using the sliding window approach.

In [19]:
# Tokenizing the whole verdict story.

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [20]:
tokenizer = SimpleTokenizerV2(vocab)
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

4690


In [21]:
# Removing the first 50 tokens for testing - results in better subsequent predictions.

enc_sample = enc_text[50:]

In [23]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [568, 115, 1066, 727]
y: [115, 1066, 727, 988]


In [24]:
# example of the next word prediction tassks.

for i in range (1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "------>", desired)

[568] ------> 115
[568, 115] ------> 1066
[568, 115, 1066] ------> 727
[568, 115, 1066, 727] ------> 988


# Implementing and efficient dataloader

In [32]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader

In [27]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
             input_chunk = token_ids[i:i + max_length]
             target_chunk = token_ids[i + 1: i + max_length + 1]
             self.input_ids.append(torch.tensor(input_chunk))
             self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [38]:
# Now we will load the inputs in batches using the PyTorch DataLoader.

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, 
                            batch_size=batch_size, 
                            shuffle=shuffle, 
                            drop_last=drop_last, 
                            num_workers=num_workers)
    return dataloader

In [39]:
# Testing with a batch size of 1 context size of 4.

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_test = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


# Creating the token Embeddings
 - Last step in preparing the input text is converting the tokekIDs into embedding vectors.

In [46]:
input_ids = torch.tensor([2, 3, 5, 1])

vocab_size = 6
output_dim = 3

In [41]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [44]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [47]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# Encoding Word Positions

In [48]:
# We encode the positional data so the model learns about the varying positions of embedding in the data.

vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [50]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, 
    max_length=max_length,
    stride = max_length, 
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [51]:
# Using the embedding layer to embed these token IDs into 256-dimensional vectors.
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [54]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input)